In [ ]:
# cifar10_pytorch_adam_small.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Reproducibility
torch.manual_seed(42)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Hyperparams
batch_size = 64
epochs = 10   # CIFAR-10 khó hơn nên train nhiều hơn MNIST
lr = 1e-3

# Data transforms (standard CIFAR-10 normalization for RGB)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2470, 0.2435, 0.2616))
])

# Datasets + DataLoaders
train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=1000, shuffle=False, num_workers=2, pin_memory=True)

# --- Smaller CNN for CIFAR-10 ---
class SmallCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 1st conv: RGB (3) -> 32 filters
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)   # 32x32 -> 32x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 32x32 -> 32x32
        self.pool  = nn.MaxPool2d(2, 2)                                     # 32x32 -> 16x16
        self.dropout1 = nn.Dropout(0.25)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Model
model = SmallCNN().to(device)
print(model)

# Loss + Optimizer (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
def train(epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx + 1) % 200 == 0:
            print(f"Train Epoch: {epoch} [{(batch_idx+1)*len(data)}/{len(train_loader.dataset)}]  Loss: {running_loss/200:.4f}")
            running_loss = 0.0

# Evaluation
def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            preds = output.argmax(dim=1, keepdim=False)
            correct += preds.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.*accuracy:.2f}%)")
    return test_loss, accuracy

# Run training
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

# Save model
torch.save(model.state_dict(), "cifar10_smallcnn_adam.pt")
print("Model saved to cifar10_smallcnn_adam.pt")

5
